# Setting up the notebook

In [ ]:
#make sure you are connected to a GPU to make use of hardware acceleration
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jul 15 12:36:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = 'cuda'
import torch, gc
import os
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
#install and import 
!pip install transformers
from transformers import BertConfig
from transformers import BertForSequenceClassification
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import get_scheduler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Read in data


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd


print("loading csv files...")
X_train = pd.read_csv('/content/gdrive/MyDrive/PANC_datasets/PANC-train_add_features_emotion.csv', header=0, encoding='latin1')
X_test = pd.read_csv('/content/gdrive/MyDrive/PANC_datasets/PANC-test_add_features_emotion.csv', header=0, encoding= 'latin1')



#just for inspection and normalization
# Add a column named 'split', and specify which split each row belongs to.
X_train['split'] = 'train'
X_test['split'] = 'test'


# Combine the two into a single dataframe for processing.
df = pd.concat([X_train, X_test])
#transform to numerical labels
df.loc[:, 'label'] = (df['label'] == 'predator').apply(int)
df



loading csv files...


,Unnamed: 0,label,chatName,segment,length_rel,length_abs,num_questions_rel,num_questions_abs,size_of_words_rel,size_of_words_abs,...,bert_nervousness,bert_optimism,bert_pride,bert_realization,bert_relief,bert_remorse,bert_sadness,bert_surprise,bert_neutral,split
0,0,0,9fbb1cac6583f9df43d968c02cfc9eff,[CLS] Hey hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii m/...,0.277439,565.0,1.000000,3.0,0.280625,139.957143,...,0.002213,0.002550,0.000968,0.007122,0.000705,0.001172,0.004991,0.008361,0.077860,train
1,1,0,afcdee26c7ba50454fe886e80eb16ded,[CLS] hi hi easter is gay how are u? yea agree...,0.572314,691.0,0.000000,4.0,0.369517,148.843651,...,0.002361,0.065858,0.004021,0.018808,0.002213,0.001050,0.003822,0.002383,0.109519,train
2,2,1,aggresk8er-0,[CLS] hello how are you hi how are you goo goo...,0.026667,1184.0,0.857143,16.0,0.260687,320.263490,...,0.000863,0.003288,0.002020,0.007620,0.001647,0.001496,0.005680,0.004913,0.017312,train
3,3,1,aggresk8er-1,[CLS] Hello hi what ya doin? Nothing much just...,0.403315,289.0,0.500000,3.0,0.255186,69.497619,...,0.000915,0.004819,0.000722,0.005034,0.000704,0.015193,0.013506,0.002184,0.022046,train
4,4,1,aggresk8er-4,[CLS] Hello hi How r u doing good what u been ...,0.016981,1051.0,1.000000,9.0,0.248014,221.249451,...,0.000613,0.012490,0.003142,0.004357,0.002666,0.000648,0.001321,0.002366,0.013954,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13154,13154,0,6377d2c072798e7ef0fd22f0769fdbae,[CLS] 15 m hey im 15 f from hey im bobby usa c...,0.131579,994.0,0.857143,8.0,0.227453,221.526190,...,0.001099,0.007855,0.001591,0.011055,0.000818,0.000697,0.002752,0.002364,0.075391,test
13155,13155,0,a2378441841247f277c54964860db8c9,[CLS] hiii hw r u hiiii are you okay ya u too ...,0.109890,172.0,0.000000,0.0,0.123126,68.850000,...,0.000959,0.002663,0.001041,0.008098,0.000736,0.000540,0.004075,0.000781,0.773637,test
13156,13156,0,497ec625519d6e28dd360be81d01dbbd,[CLS] Why should interruptions be more problem...,0.811240,3067.0,0.333333,5.0,0.808098,283.292332,...,0.002674,0.010463,0.002572,0.024410,0.001482,0.001058,0.005327,0.004805,0.375396,test
13157,13157,0,dfbec532af50f894e0961bb5ef935c4c,[CLS] boo aaaaaaaaaah :o you scared me Yes I s...,0.060071,549.0,0.750000,5.0,0.315303,137.597619,...,0.006449,0.006909,0.005956,0.014781,0.020089,0.002330,0.009367,0.004675,0.026990,test


In [ ]:
#specify our numerical features
all_columns = df.columns.values.tolist()

print(all_columns)

col_info = {}
col_info['text_cols'] = [
                         "chatName",
                         "segment",
]
col_info['num_cols'] = [
             "length_rel",
             "length_abs",
             "num_questions_rel",
             "num_questions_abs",
             "size_of_words_rel",
             "size_of_words_abs",
             'empath_help', 'empath_office', 'empath_dance', 'empath_money', 'empath_wedding', 'empath_domestic_work', 'empath_sleep', 'empath_medical_emergency', 'empath_cold', 'empath_hate', 'empath_cheerfulness', 'empath_aggression', 'empath_occupation', 'empath_envy', 'empath_anticipation', 
             'bert_admiration', 'bert_amusement', 'bert_anger', 'bert_annoyance', 'bert_approval', 'bert_caring', 'bert_confusion', 'bert_curiosity', 'bert_desire', 'bert_disappointment', 'bert_disapproval', 'bert_disgust', 'bert_embarrassment', 'bert_excitement', 'bert_fear', 'bert_gratitude', 'bert_grief', 'bert_joy', 'bert_love', 'bert_nervousness', 'bert_optimism', 'bert_pride', 'bert_realization', 'bert_relief', 'bert_remorse', 'bert_sadness', 'bert_surprise', 'bert_neutral',
            ]


print('There are:')
print('  {:,}  numerical features'.format(len(col_info['num_cols'])))

# Select just the columns we're using.
data_df = df[col_info['text_cols'] +  
             col_info['num_cols'] +
             ['label', 'split']] # 'label' is our label.
              

data_df.head()
data_df = data_df.fillna("")

['Unnamed: 0', 'label', 'chatName', 'segment', 'length_rel', 'length_abs', 'num_questions_rel', 'num_questions_abs', 'size_of_words_rel', 'size_of_words_abs', 'empath_help', 'empath_office', 'empath_dance', 'empath_money', 'empath_wedding', 'empath_domestic_work', 'empath_sleep', 'empath_medical_emergency', 'empath_cold', 'empath_hate', 'empath_cheerfulness', 'empath_aggression', 'empath_occupation', 'empath_envy', 'empath_anticipation', 'bert_admiration', 'bert_amusement', 'bert_anger', 'bert_annoyance', 'bert_approval', 'bert_caring', 'bert_confusion', 'bert_curiosity', 'bert_desire', 'bert_disappointment', 'bert_disapproval', 'bert_disgust', 'bert_embarrassment', 'bert_excitement', 'bert_fear', 'bert_gratitude', 'bert_grief', 'bert_joy', 'bert_love', 'bert_nervousness', 'bert_optimism', 'bert_pride', 'bert_realization', 'bert_relief', 'bert_remorse', 'bert_sadness', 'bert_surprise', 'bert_neutral', 'split']
There are:
  49  numerical features


In [ ]:
data_df[col_info['num_cols']].head()

,length_rel,length_abs,num_questions_rel,num_questions_abs,size_of_words_rel,size_of_words_abs,empath_help,empath_office,empath_dance,empath_money,...,bert_love,bert_nervousness,bert_optimism,bert_pride,bert_realization,bert_relief,bert_remorse,bert_sadness,bert_surprise,bert_neutral
0,0.277439,565.0,1.0,3.0,0.280625,139.957143,0.0,0.0,0.0,0.0,...,0.000419,0.002213,0.002550,0.000968,0.007122,0.000705,0.001172,0.004991,0.008361,0.077860
1,0.572314,691.0,0.0,4.0,0.369517,148.843651,0.0,0.0,0.0,0.0,...,0.002123,0.002361,0.065858,0.004021,0.018808,0.002213,0.001050,0.003822,0.002383,0.109519
2,0.026667,1184.0,0.857143,16.0,0.260687,320.26349,0.0,0.0,1.0,0.0,...,0.002156,0.000863,0.003288,0.002020,0.007620,0.001647,0.001496,0.005680,0.004913,0.017312
3,0.403315,289.0,0.5,3.0,0.255186,69.497619,1.0,0.0,0.0,0.0,...,0.001936,0.000915,0.004819,0.000722,0.005034,0.000704,0.015193,0.013506,0.002184,0.022046
4,0.016981,1051.0,1.0,9.0,0.248014,221.249451,0.0,0.0,1.0,0.0,...,0.001744,0.000613,0.012490,0.003142,0.004357,0.002666,0.000648,0.001321,0.002366,0.013954


In [ ]:
#normalize feature distribution to help MLP
from sklearn.preprocessing import QuantileTransformer

for x in col_info['num_cols']:
    data_df[x].replace('', 0, inplace=True)
    col_val = data_df[x].values.reshape(-1, 1)

    numerical_transformer = QuantileTransformer(output_distribution='normal')
    

    data_df[x] = numerical_transformer.fit_transform(col_val)

    
data_df[col_info['num_cols']].head()


,length_rel,length_abs,num_questions_rel,num_questions_abs,size_of_words_rel,size_of_words_abs,empath_help,empath_office,empath_dance,empath_money,...,bert_love,bert_nervousness,bert_optimism,bert_pride,bert_realization,bert_relief,bert_remorse,bert_sadness,bert_surprise,bert_neutral
0,-0.341501,0.139710,5.199338,-0.634851,-0.078946,0.250185,-5.199338,-5.199338,-5.199338,-5.199338,...,-1.905080,0.449730,-1.614523,-0.902874,-0.978498,-0.702830,0.355550,0.458676,0.670574,-0.322039
1,0.665623,0.352151,-5.199338,-0.427976,0.252356,0.330791,-5.199338,-5.199338,-5.199338,-5.199338,...,0.126973,0.528085,1.859563,0.944741,0.589005,0.686556,0.217900,0.191087,-0.914693,-0.089193
2,-1.745266,0.894843,0.494971,1.081940,-0.157644,1.233666,-5.199338,-5.199338,5.199338,-5.199338,...,0.143349,-0.848026,-1.217205,0.064464,-0.874907,0.386868,0.606537,0.575667,0.062123,-1.434942
3,0.097195,-0.697507,-0.204660,-0.634851,-0.179156,-1.013596,5.199338,-5.199338,-5.199338,-5.199338,...,0.030505,-0.755930,-0.603514,-1.348888,-1.569547,-0.705494,1.656367,1.144054,-1.041340,-1.242587
4,-1.887577,0.773986,5.199338,0.388430,-0.206617,0.808818,-5.199338,-5.199338,5.199338,-5.199338,...,-0.083118,-1.431763,0.672838,0.620307,-1.800414,0.851644,-0.543360,-1.361949,-0.924818,-1.621891


In [ ]:
# Re-split the dataset.
test_df = data_df.loc[data_df.split == 'test']
train_df = data_df.loc[data_df.split == 'train']

datasets = {'train': train_df, 'eval': test_df}


# Helper functions: MLP

This is a standard MLP, which is used on the CLS embeddings of BERT's output to generate our final predictions with the feature vectors. 
The MLP was taken directly from the Multimodal-Toolkit [here](https://github.com/georgian-io/Multimodal-Toolkit/blob/master/multimodal_transformers/model/layer_utils.py)


In [ ]:
import math
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss


class MLP(nn.Module):
    """mlp can specify number of hidden layers and hidden layer channels"""

    def __init__(self, input_dim, output_dim, num_hidden_lyr=2,
                 dropout_prob=0.5, return_layer_outs=False,
                 hidden_channels=None, bn=False):
        super().__init__()
        self.out_dim = output_dim
        self.dropout = nn.Dropout(dropout_prob)
        self.return_layer_outs = return_layer_outs
        if not hidden_channels:
            hidden_channels = [input_dim for _ in range(num_hidden_lyr)]
        elif len(hidden_channels) != num_hidden_lyr:
            raise ValueError(
                "number of hidden layers should be the same as the lengh of hidden_channels")
        self.layer_channels = [input_dim] + hidden_channels + [output_dim]
        self.act_name = 'relu'
        self.activation = nn.ReLU()
        self.layers = nn.ModuleList(list(
            map(self.weight_init, [nn.Linear(self.layer_channels[i], self.layer_channels[i + 1])
                                   for i in range(len(self.layer_channels) - 2)])))
        final_layer = nn.Linear(self.layer_channels[-2], self.layer_channels[-1])
        self.weight_init(final_layer,   activation='linear')
        self.layers.append(final_layer)

        self.bn = bn
        if self.bn:
            self.bn = nn.ModuleList([torch.nn.BatchNorm1d(dim) for dim in self.layer_channels[1:-1]])

    def weight_init(self, m, activation=None):
        if activation is None:
            activation = self.act_name
        torch.nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain(activation))
        return m

    def forward(self, x):
        """
        :param x: the input features
        :return: tuple containing output of MLP,
                and list of inputs and outputs at every layer
        """
        layer_inputs = [x]
        for i, layer in enumerate(self.layers):
            input = layer_inputs[-1]
            if layer == self.layers[-1]:
                layer_inputs.append(layer(input))
            else:
                if self.bn:
                    output = self.activation(self.bn[i](layer(input)))
                else:
                    output = self.activation(layer(input))
                layer_inputs.append(self.dropout(output))

        # model.store_layer_output(self, layer_inputs[-1])
        if self.return_layer_outs:
            return layer_inputs[-1], layer_inputs
        else:
            return layer_inputs[-1]

In [ ]:
#Precalculate dimensions of MLP (numbers are manually added in the Modified BERT class)
# According to the formula in the MultiModal-Toolkit, each layer of the MLP has 1/4th the number of neurons as the previous one. 
dims = []
dim = 768 + 49 #hidden size of bert cls + number of features
# Keep dividing by 4
while True:
    # Divide by 4 and truncate to an integer.
    dim = dim // 4
    
    # until resulting layer is smaller than #outputs. Since we have a classification task, that's 2
    if dim <= 2:
        break
    # store as next layer
    dims.append(int(dim))

# Print out the resulting MLP.                
print('MLP layer sizes:')
print('Input:', dim)
print('Hidden:', dims)
print('Output:', 2)

#Results: Input: 817
          #Hidden: [204, 51, 12, 3]
          #Output: 2




MLP layer sizes:
Input: 0
Hidden: [204, 51, 12, 3]
Output: 2


# Helper functions: Modified BERT

This class modifies the forward function of BertForSequenceClassification in order to pass the CLS embeddings of the output + the feature vector through an MLP

In [ ]:
from torch import nn

from transformers import BertForSequenceClassification


class BertModified(BertForSequenceClassification):
    """
    Uses a transformers.BertConfig object which has two additional properties manually added to it:
      `text_feat_dim` - The length of the BERT vector (usually 768)
      `numerical_feat_dim` - The number of numerical features (in our case 49)
    """

    def __init__(self, config):
        # Calling constructor of the`BertForSequenceClassification` class, does all of the BERT-related setup. Resulting BERT model is stored in `self.bert`
        super().__init__(config)
        # Create a batch normalizer for the numerical features.
        # Calculate the combined vector length. 
        combined_feat_dim = config.text_feat_dim + config.numerical_feat_dim
        self.num_bn = nn.BatchNorm1d(config.numerical_feat_dim)
        #As previously calculated, specify the number of inputs, outputs, and the
        # layer sizes when constructing the MLP.
        self.mlp = MLP(combined_feat_dim, #text features dim + numerical features dim
                        2, #number of labels
                        num_hidden_lyr=4, #len(dims)
                        dropout_prob=0.1,
                        hidden_channels= [204, 51, 12, 3], #dims
                        bn=True)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        class_weights=None,
        output_attentions=None,
        output_hidden_states=None,
        numerical_feats=None
    ):
        """
        forward pass of our model with the same inputs as `forward` in `BertForSequenceClassification`
        + one extra parameter: `numerical_feats` = Tensor of numerical features.  
        """
        #`self.bert` returns outputs from the encoding layers, and not from the final classifier.
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        
        # outputs[0] - All of the output embeddings from BERT
        # outputs[1] - Apparently The [CLS] token embedding
        cls = outputs[1] #should be of size [batch size  x   768]
       # Apply batch normalizat ion to the numerical features.        
        numerical_feats = self.num_bn(numerical_feats) #should be of size [batch size  x   # numerical features]

        #concatenate into one vector
        combined_feats = torch.cat((cls, numerical_feats),
                                    dim=1)
     
        # Run the the samples through the MLP.
        logits = self.mlp(combined_feats)

        if type(logits) is tuple:
            logits = logits[0]

        # If we're doing training or validation, calculate loss with the provided labels
        if labels is not None:
            loss_fct = CrossEntropyLoss(weight=class_weights)
            labels = labels.long()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))
        
        # During test time, we don't need labels.
        else:
            loss = None
        
        # Put the results into a Dictionary to return.
        results = {'loss': loss, 
                   'logits': logits}

        return results




# Training class

In [ ]:
from transformers import BertTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
import torch
from torch.utils.data import TensorDataset, random_split
import torch

class ESPD:
    '''
    Implementation of the Early Sexual Predator Detection paper
    '''

    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
    
        self.config = BertConfig.from_pretrained("bert-base-cased",
                                    output_hidden_states=True, num_labels = 2) #for classification
        self.num_epochs = 4
        self.training_stats = []

    def tokenize_function(self, text_list):
        print('Encoding {:,} text samples...'.format(len(text_list)))
        input_ids = []
        attention_masks = []
        for row in text_list:
            encoded_dict = self.tokenizer.encode_plus(
                                row,                  # Sentence to encode.
                                truncation = True,
                                padding = 'max_length',
                                pad_to_max_length = True, # Pad & truncate all sentences.
                                return_attention_mask = True,   # Construct attn. masks.
                                return_tensors = 'pt',     # Return pytorch tensors.
                        )
            
            # Add the encoded sentence to the list.    
            input_ids.append(encoded_dict['input_ids'])
            
            # And its attention mask (simply differentiates padding from non-padding).
            attention_masks.append(encoded_dict['attention_mask'])
        print("done")
        return input_ids, attention_masks

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return self.metric.compute(predictions=predictions, references=labels)
    

    def split(self, dataset):
        # Create a 90-10 train-validation split.
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size

        # Divide the dataset by randomly selecting samples.
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        return train_dataset, val_dataset




    def create_tensorData(self, input_ids_t, attention_masks_t, features_t, labels_t):

        # Convert the lists into tensors.
        input_ids = torch.cat(input_ids_t, dim=0)
        attention_masks = torch.cat(attention_masks_t, dim=0)
        labels = torch.tensor(labels_t.values)
        numerical_feats = torch.tensor(features_t.values.astype('float'), dtype=torch.float32)
        #finish setting up Config and the actual model
        self.config.numerical_feat_dim = numerical_feats.size()[1]

        data = TensorDataset(input_ids, attention_masks, labels, numerical_feats)

        return data


    def preprocess_dataset(self, dataset):
        texts_train, labels_train, features_train = dataset['train']['segment'], dataset['train']['label'], dataset['train'][col_info['num_cols']]
        texts_test, labels_test, features_test =  dataset['eval']['segment'], dataset['eval']['label'], dataset['eval'][col_info['num_cols']]


        texts_train.map(lambda x: x.replace('[CLS] ', ''))
        texts_test.map(lambda x: x.replace('[CLS] ', ''))

        input_ids_train, attention_masks_train = self.tokenize_function(texts_train.tolist())
        input_ids_test, attention_masks_test = self.tokenize_function(texts_test.tolist())

        train_and_val_data = self.create_tensorData(input_ids_train, attention_masks_train, features_train, labels_train)
        test_dataset = self.create_tensorData(input_ids_test, attention_masks_test, features_test, labels_test)

        #split into train and validation set
        print("creating a validation set out of training data...")
        train_dataset, val_dataset = self.split(train_and_val_data)

        train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle=True, # Select batches randomly
            batch_size = 16 # Trains with this batch size.
        )

        # For validation the order doesn't matter, so we'll just read them sequentially.
        validation_dataloader = DataLoader(
                    val_dataset, # The validation samples.
                    sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                    batch_size = 16 # Evaluate with this batch size.
        )

        test_dataloader = DataLoader(
            test_dataset, 
            sampler=SequentialSampler(test_dataset), 
            batch_size=16)

        
        #finish setting up the model
        self.config.text_feat_dim = self.config.hidden_size # 768
        self.model = BertModified.from_pretrained("bert-base-cased", config = self.config)
        # Tell pytorch to run this model on the GPU.
        desc = self.model.cuda()

        self.optimizer = AdamW(self.model.parameters(), lr = 3e-3, eps = 1e-8)
        self.num_training_steps = self.num_epochs * len(train_dataloader)
        self.lr_scheduler = get_scheduler(name="linear", optimizer=self.optimizer, num_warmup_steps=0, num_training_steps=self.num_training_steps)

        print("returning dataloaders...")

        return train_dataloader, validation_dataloader, test_dataloader


    def start_training(self, train_dataloader, validation_dataloader):
          for epoch_i in range(0, self.num_epochs):
                    print("")
                    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, self.num_epochs))
                    print('Training...')
                    # Reset the total loss for this epoch.
                    total_train_loss = 0
                    #set model in training mode (doesn't train yet, is just a mode)
                    self.model.train()
                    for i, batch in enumerate(train_dataloader):
                          if i % 40 == 0 and not i == 0:
                            print('  Batch {:>5,}  of  {:>5,}.'.format(i, len(train_dataloader)))
                          #Unpack this training batch from our dataloader and copy each tensor to device
                          b_input_ids = batch[0].to(device)
                          b_input_mask = batch[1].to(device)
                          b_labels = batch[2].to(device)
                          b_numerical_feats = batch[3].to(device)

                          #clear any previously calculated gradients before performing backward pass
                          self.model.zero_grad()

                          #forward pass
                          result = self.model(b_input_ids,
                                      token_type_ids=None,
                                      attention_mask=b_input_mask,
                                      labels=b_labels,
                                      numerical_feats = b_numerical_feats)

                          loss = result['loss']

                          total_train_loss += loss.item() #returns python value from tensor

                          #backward pass
                          loss.backward()

                          #clip norm of gradients to 1 so that they dont explode
                          torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                          #update rule
                          self.optimizer.step()
                          # Update the learning rate.
                          self.lr_scheduler.step()
                    
                    avg_train_loss = total_train_loss / (len(train_dataloader) * 16)
                    print("")
                    print("  Average training loss: {0:.2f}".format(avg_train_loss))
                    print("")
                    print("Running Validation...")
                    #change mode of evaluation
                    self.model.eval()
                    total_eval_loss = 0
                    for i, batch in enumerate(validation_dataloader):
                            if i % 40 == 0 and not i == 0:
                              print('  Batch {:>5,}  of  {:>5,}.'.format(i, len(validation_dataloader)))
                            #Unpack this training batch from our dataloader and copy each tensor to device
                            b_input_ids = batch[0].to(device)
                            b_input_mask = batch[1].to(device)
                            b_labels = batch[2].to(device)
                            b_numerical_feats = batch[3].to(device)
          
                            #since this is validation
                            with torch.no_grad():
                              #forward pass
                              result = self.model(b_input_ids,
                                            token_type_ids=None,
                                            attention_mask=b_input_mask,
                                            labels=b_labels,
                                            numerical_feats = b_numerical_feats)
          
                              loss = result['loss']
                        
                              total_eval_loss += loss.item()
                          
                      # Calculate the average loss over all of the batches.
                    avg_val_loss = total_eval_loss / (len(validation_dataloader) * 16)
                    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
                    # Record all statistics from this epoch.
                    self.training_stats.append(
                            {
                              'epoch': epoch_i + 1,
                              'Training Loss': avg_train_loss,
                              'Valid. Loss': avg_val_loss
                            })
          torch.save(self.model, '/content/gdrive/MyDrive/mzModel2')







      

    

# Main Calls to classes

In [ ]:
eSPD = ESPD()
train_dataloader, validation_dataloader, test_dataloader = eSPD.preprocess_dataset(datasets)

Encoding 17,415 text samples...
done
Encoding 13,159 text samples...
done
creating a validation set out of training data...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModified: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModified from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModified from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModified were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['num_bn.weight', 'mlp.lay

returning dataloaders...


In [ ]:
import random
#for reproducability
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.autograd.set_detect_anomaly(True)
eSPD.start_training(train_dataloader, validation_dataloader)


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    980.
  Batch    80  of    980.
  Batch   120  of    980.
  Batch   160  of    980.
  Batch   200  of    980.
  Batch   240  of    980.
  Batch   280  of    980.
  Batch   320  of    980.
  Batch   360  of    980.
  Batch   400  of    980.
  Batch   440  of    980.
  Batch   480  of    980.
  Batch   520  of    980.
  Batch   560  of    980.
  Batch   600  of    980.
  Batch   640  of    980.
  Batch   680  of    980.
  Batch   720  of    980.
  Batch   760  of    980.
  Batch   800  of    980.
  Batch   840  of    980.
  Batch   880  of    980.
  Batch   920  of    980.
  Batch   960  of    980.

  Average training loss: 0.02

Running Validation...
  Batch    40  of    109.
  Batch    80  of    109.
  Validation Loss: 0.01

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    980.
  Batch    80  of    980.
  Batch   120  of    980.
  Batch   160  of    980.
  Batch   200  of    980.
  Batch   240  of    980.
  Ba

# Testing

In [ ]:
print('Predicting labels for test sentences...')

# Put model in evaluation mode
eSPD.model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_numer_feats = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = eSPD.model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     numerical_feats = b_numer_feats)


  logits = result['logits']

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for test sentences...
    DONE.


In [ ]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

pred_scores = softmax(flat_predictions, axis=1)[:, 1]

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1)



# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)



roc_auc_pred_score = roc_auc_score(flat_true_labels, pred_scores)
precisions, recalls, thresholds = precision_recall_curve(flat_true_labels, pred_scores)
fscore = (2 * precisions * recalls) / (precisions + recalls)
fscore[np.isnan(fscore)] = 0
ix = np.argmax(fscore)
threshold = thresholds[ix].item()
pr_auc = auc(recalls, precisions)
tn, fp, fn, tp = confusion_matrix(flat_true_labels, flat_predictions, labels=[0, 1]).ravel()
result = {'roc_auc': roc_auc_pred_score,
          'threshold': threshold,
          'pr_auc': pr_auc,
          'recall': recalls[ix].item(),
          'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
          'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
          }

In [ ]:
print(result)

{'roc_auc': 0.9269770091406156, 'threshold': 0.3198927938938141, 'pr_auc': 0.7003751321202794, 'recall': 0.7230014025245441, 'precision': 0.6086186540731995, 'f1': 0.6608974358974359, 'tn': 11733, 'fp': 0, 'fn': 1426, 'tp': 0}
